In [2]:
import torch
from torch import nn,optim
from torch.nn import functional as F
import numpy as np

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    
    def forward(self,x):
        return self.out(F.relu(self.hidden(x)))
        

In [7]:
x=torch.rand(2,20)

In [8]:
net=MLP()
net(x)

tensor([[-0.0175,  0.0446,  0.1960, -0.0251, -0.0335,  0.2122,  0.1869,  0.0912,
          0.0702, -0.1029],
        [-0.0106,  0.1523,  0.2236,  0.0070, -0.0758,  0.2140,  0.0470,  0.1852,
          0.0546, -0.0648]], grad_fn=<AddmmBackward>)

In [9]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)]=module
    
    def forward(self,x):
        for block in self._modules.values():
            x=block(x)
        return x

In [10]:
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(x)

tensor([[ 0.1906,  0.0762, -0.1163, -0.1307,  0.1659,  0.0450, -0.1994, -0.1290,
          0.0978,  0.0399],
        [ 0.1450,  0.0849, -0.2085, -0.1274,  0.1808,  0.1111, -0.2462, -0.1340,
          0.0860,  0.0065]], grad_fn=<AddmmBackward>)

In [13]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
        
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.matmul(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum()>1 :
            X /=2
        return X.sum()
        
        

In [14]:
net=FixedHiddenMLP()
net(x)

tensor(0.0716, grad_fn=<SumBackward0>)

In [15]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())
        self.Linear=nn.Linear(32,16)
    
    def forward(self,X):
        return self.Linear(self.net(X))

In [17]:
chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(x)

tensor(-0.2639, grad_fn=<SumBackward0>)